In [1]:
import torch
from cosypose.config import LOCAL_DATA_DIR
from cosypose.datasets.datasets_cfg import make_scene_dataset
from cosypose.rendering.bullet_scene_renderer import BulletSceneRenderer
from cosypose.visualization.singleview import make_singleview_prediction_plots, filter_predictions
from cosypose.visualization.singleview import filter_predictions
from bokeh.plotting import gridplot
from bokeh.io import show, output_notebook; output_notebook()
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# From visualisation.singleview.py
from cosypose.datasets.wrappers.augmentation_wrapper import AugmentationWrapper
from cosypose.datasets.augmentations import CropResizeToAspectAugmentation
import numpy as np
from cosypose.visualization.plotter import Plotter


Setting OMP and MKL num threads to 1.


Loading BokehJS ...

In [2]:
def disp_predictions(scene_id, view_id, scene_ds):
    this_preds = filter_predictions(results[pred_key], scene_id, view_id)
    renderer = BulletSceneRenderer(urdf_ds_name)

    # figures = make_singleview_prediction_plots(scene_ds, renderer, this_preds)
    figures = None
    predictions = this_preds
    detections = None
    resolution = (640, 480)
    if True:
        plotter = Plotter()

        scene_id, view_id = np.unique(predictions.infos['scene_id']).item(), np.unique(predictions.infos['view_id']).item()

        scene_ds_index = scene_ds.frame_index
        scene_ds_index['ds_idx'] = np.arange(len(scene_ds_index))
        scene_ds_index = scene_ds_index.set_index(['scene_id', 'view_id'])
        idx = scene_ds_index.loc[(scene_id, view_id), 'ds_idx']

        augmentation = CropResizeToAspectAugmentation(resize=resolution)
        scene_ds = AugmentationWrapper(scene_ds, augmentation)
        rgb_input, mask, state = scene_ds[idx]

        figures = dict()
        figures['input_im'] = plotter.plot_image(rgb_input)

        if detections is not None:
            fig_dets = plotter.plot_image(rgb_input)
            fig_dets = plotter.plot_maskrcnn_bboxes(fig_dets, detections)
            figures['detections'] = fig_dets

        # pred_rendered = render_prediction_wrt_camera(renderer, predictions, camera=state['camera'])
        # render_predictions_with_respect_to_camera
        pred_rendered = []
        pred = predictions
        renderer = renderer
        camera = state['camera']
        if True:
            pred = pred.cpu()
            camera.update(TWC=np.eye(4))

            list_objects = []
            for n in range(len(pred)):
                row = pred.infos.iloc[n]
                obj = dict(
                    name=row.label,
                    color=(1, 1, 1, 1),
                    TWO=pred.poses[n].numpy(),
                )
                list_objects.append(obj)
            # looks like this place is not required for measuring distances
            pred_rendered = renderer.render_scene(list_objects, [camera])[0]['rgb']
        figures['pred_rendered'] = plotter.plot_image(pred_rendered)
        figures['pred_overlay'] = plotter.plot_overlay(rgb_input, pred_rendered)
        #return figures

    renderer.disconnect()
    print(this_preds)

    show(figures['input_im'])
    show(figures['pred_overlay'])

print(f"LocalDataDir = {LOCAL_DATA_DIR}")

result_id = 'tless-siso-n_views=1--684390594'
ds_name, urdf_ds_name = 'tless.primesense.test', 'tless.cad'
pred_key = 'pix2pose_detections/refiner/iteration=4' # we have 4 iterations available

# result_id = 'ycbv-n_views=1--5154971130'
# ds_name, urdf_ds_name = 'ycbv.test.keyframes', 'ycbv'
# pred_key = 'posecnn_init/refiner/iteration=2'


results = LOCAL_DATA_DIR / 'results' / result_id / 'results.pth.tar'
scene_ds = make_scene_dataset(ds_name)
results = torch.load(results)['predictions']

print(results.keys())

print("Available scenes:")
print(results[pred_key].infos.loc[:, ['scene_id', 'view_id']].groupby('scene_id').first())

disp_predictions(5, 27, scene_ds)


0:00:00.140455 - Building index and loading annotations...


LocalDataDir = /home/dkushn/cosypose/local_data
odict_keys(['pix2pose_detections/coarse/iteration=1', 'pix2pose_detections/refiner/iteration=1', 'pix2pose_detections/refiner/iteration=2', 'pix2pose_detections/refiner/iteration=3', 'pix2pose_detections/refiner/iteration=4'])
Available scenes:
          view_id
scene_id         
1             439
2             307
3             353
4              27
5             330
6             170
7             412
8             300
9             190
10            439
11             69
12            103
13            125
14            294
15            337
16            327
17            452
18            108
19             83
20             58
PandasTensorCollection(
    poses: torch.Size([5, 4, 4]) torch.float32 cpu,
    poses_input: torch.Size([5, 4, 4]) torch.float32 cpu,
    K_crop: torch.Size([5, 3, 3]) torch.float32 cpu,
    boxes_rend: torch.Size([5, 4]) torch.float32 cpu,
    boxes_crop: torch.Size([5, 4]) torch.float32 cpu,
    initial_bbox